# 신경망 학습

학습이란 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 흭득하는 것을 의미한다.

기계학습 문제는 데이터를 **훈련 데이터(training data)**와 **시험 데이터(test data)**로 나눠 학습과 시험을 수행하는 것이 일반적입니다.

보통의 진행은 훈련 데이터만 사용하여 학습하면서 최적의 매개변수를 찾습니다. 그 후 시험 데이터를 사용하여 앞서 훈련한 모델의 실력을 평가합니다.

이를 하는 이유는 **범용 능력**을 제대로 평가하기 위함입니다.
**범용 능력** : 학습 하지 못한 데이터도 올바르게 푸는 능력

하지만 그렇다고 한 데이터셋으로만 지나치게 최적화 되면 **오버피팅**이라는 문제가 생깁니다.